In [27]:
Bf_z = list( range(50,550,50))
import arcpy
#aprx_path = r'X:\Projects\2019\ENV.533.aprx'
#gdb_path = r'X:\shared\ENV_default.gdb'
from arcpy import env
env.parallelProcessingFactor = "0"
env.overwriteOutput = True
env.cellSize = 0.2
env.addOutputsToMap = False
import pandas as pd
import numpy as np
aprx = arcpy.mp.ArcGISProject('CURRENT') # or 'CURRENT' in live py console
aprx.defaultGeodatabase
env.addOutputsToMap = False

In [28]:
INT = "Brokke_IT_Dam"
Rs_ch = "Broke_T1_ComputeChangeRa.crf"
OP_N = "Brokke_IT"

In [29]:
#Buffers 
fields = ['OID@']# OID@ is ObjectID token (see search cursor help)
# clear any existing selections
arcpy.SelectLayerByAttribute_management(f'{INT}', "CLEAR_SELECTION")
#Select Feature 
cursor = arcpy.da.SearchCursor(f'{INT}', fields)
for FID in  (cursor): # unpack each row by assigning to field name variables
        T = FID[0]
        print(T)
        arcpy.management.SelectLayerByAttribute(f'{INT}', "NEW_SELECTION", f"OBJECTID = {FID[0]}", None)
        for j in Bf_z:
            #print (list(T,j))
            arcpy.analysis.Buffer(f'{INT}', 
                            "Buf", f"{j} Meters", "FULL", "ROUND", "NONE", None, "PLANAR")
#Get the Area for each interval 
            print(j)
            o = arcpy.sa.ExtractByMask(f"{Rs_ch}", "Buf")
            o.save(rf"CC_int_{T}_{j}") 
            arcpy.management.BuildRasterAttributeTable(rf"CC_int_{T}_{j}", "Overwrite", "NONE")
            arcpy.management.JoinField(rf"CC_int_{T}_{j}", "Value",
                                       f"{Rs_ch}", 
                               "Value", "Class_name;Class_From;Class_To")
            arcpy.management.CalculateField(rf"CC_int_{T}_{j}", "Dam", f"{T}", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
            arcpy.management.CalculateField(rf"CC_int_{T}_{j}", "Buffer_Zone", f"{j}", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
            arcpy.management.Delete("Buf", '')

1
50


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 010568: Invalid extent. Please check for zero length or width, or failure to project extent to output spatial reference. 
Failed to execute (ExtractByMask).


In [11]:
R1 = arcpy.ListRasters("CC_int_*_*")
len(R1)

60

In [13]:
#Just for reWriting my List DONT USE
Area_bf = []
for i in R1:
   df_CC= pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(i,('Class_From', 'Class_To','Count',"Buffer_Zone","Dam")))
   Area_bf.append(df_CC) 
len(Area_bf)

60

In [14]:
for i in Area_bf : 
    i["Area_KM2"] = i['Count']*.2*.2

In [15]:
Area_bf 

[  Class_From Class_To     Count Buffer_Zone Dam  Area_KM2
0          1        5    3588.0          50   6    143.52
1          5        1   71201.0          50   6   2848.04
2       Same     Same  121101.0          50   6   4844.04,   Class_From Class_To     Count Buffer_Zone Dam  Area_KM2
0          1        5    6854.0         100   6    274.16
1          5        1  240486.0         100   6   9619.44
2          5        2     659.0         100   6     26.36
3       Same     Same  536613.0         100   6  21464.52,   Class_From Class_To      Count Buffer_Zone Dam  Area_KM2
0          1        5    14036.0         150   6    561.44
1          2        1      114.0         150   6      4.56
2          2        5     1567.0         150   6     62.68
3          5        1   506456.0         150   6  20258.24
4          5        2      767.0         150   6     30.68
5       Same     Same  1243184.0         150   6  49727.36,   Class_From Class_To      Count Buffer_Zone Dam  Area_KM2
0 

In [16]:
Area_BF = pd.concat(Area_bf)
Area_BF

,Class_From,Class_To,Count,Buffer_Zone,Dam,Area_KM2
0,1,5,3588.0,50,6,143.52
1,5,1,71201.0,50,6,2848.04
2,Same,Same,121101.0,50,6,4844.04
0,1,5,6854.0,100,6,274.16
1,5,1,240486.0,100,6,9619.44
...,...,...,...,...,...,...
5,2,5,57083.0,500,5,2283.32
6,5,0,118667.0,500,5,4746.68
7,5,1,5523219.0,500,5,220928.76
8,5,2,7055.0,500,5,282.20


In [17]:
Area_BF['Buffer_Zone'] = Area_BF['Buffer_Zone'].astype(str).astype(int)
Area_BF.dtypes

Class_From      object
Class_To        object
Count          float64
Buffer_Zone      int32
Dam             object
Area_KM2       float64
dtype: object

In [18]:
Area_BF.dtypes
#Filtering Deforestiation : 1 to 0,2,4,5 
Area_Def =Area_BF[(Area_BF["Class_From"] == "1")  & (Area_BF["Class_To"].isin(["0","2","4","5"]))] 
#Filtering Urbanization : All to 0 
Area_Urb =Area_BF[Area_BF["Class_To"]=="0"]
#Filtering Afforestioan 
Area_AFF = Area_BF[Area_BF["Class_To"]=="1"]
#changed Area 
#Area_Unch = Area_BF[Area_BF["Class_To"]=="Same"]


In [19]:
Are_Def_Sum = Area_Def.groupby(["Dam","Buffer_Zone"]).sum()
Are_Urb = Area_Urb.groupby(["Dam","Buffer_Zone"]).sum()
Are_AF_Sum = Area_AFF.groupby(["Dam","Buffer_Zone"]).sum()
#Are_Un_Sum = Area_Unch.groupby("Buffer_Zone").sum()

In [20]:
#Net Deforestioan 
Area_Net_Def = pd.DataFrame(Are_Def_Sum["Area_KM2"]-Are_AF_Sum["Area_KM2"])
Area_Net_Def

Area_KM2
Dam Buffer_Zone           
1   50             2971.28
    100           10822.84
    150           19596.16
    200           31783.04
    250           50650.12
    300           65325.72
    350           80794.32
    400          101390.24
    450          131542.80
    500          168094.76
2   50             2967.24
    100            9032.60
    150           12052.48
    200           23268.64
    250           38649.92
    300           46672.12
    350           53432.84
    400           71538.32
    450          100327.68
    500          126002.64
3   50            -2551.48
    100           -8563.28
    150          -11487.68
    200          -17235.08
    250          -36062.16
    300          -50757.36
    350          -74589.56
    400          -98247.32
    450         -127218.52
    500         -156050.64
4   50              786.84
    100           -1760.80
    150           -6306.32
    200          -17558.52
    250          -30611.28
    300          -49101.24
    350          -69042.68
    400          -89211.68
    450         -117316.76
    500         -143613.12
5   50            -2714.16
    100           -9358.68
    150          -19723.12
    200          -35514.72
    250          -53863.16
    300          -72106.88
    350         -101340.60
    400         -133380.04
    450         -169058.16
    500         -210129.80
6   50            -2704.52
    100           -9345.28
    150          -19701.36
    200          -35512.80
    250          -53849.44
    300          -72101.40
    350         -101346.68
    400         -133381.20
    450         -169073.68
    500         -210153.04

In [21]:

Are_Def_Sum['Area_Dif_DEF'] = Are_Def_Sum.groupby(level=0)["Area_KM2"].diff()
Are_Urb['Area_Dif_U'] = Are_Urb["Area_KM2"].groupby(level=0).diff()
Area_Net_Def['Area_Dif_NDEF'] = Area_Net_Def.groupby(level=0)["Area_KM2"].diff()

In [22]:
Are_Urb

Count  Area_KM2  Area_Dif_U
Dam Buffer_Zone                                
1   50            13437.0    537.48         NaN
    100           26012.0   1040.48      503.00
    150           31278.0   1251.12      210.64
    200           35887.0   1435.48      184.36
    250           39855.0   1594.20      158.72
    300           43848.0   1753.92      159.72
    350           50985.0   2039.40      285.48
    400           55188.0   2207.52      168.12
    450           59527.0   2381.08      173.56
    500           63395.0   2535.80      154.72
2   50            33256.0   1330.24         NaN
    100           64640.0   2585.60     1255.36
    150           84974.0   3398.96      813.36
    200          113644.0   4545.76     1146.80
    250          134443.0   5377.72      831.96
    300          147504.0   5900.16      522.44
    350          156325.0   6253.00      352.84
    400          168377.0   6735.08      482.08
    450          190509.0   7620.36      885.28
    500          209990.0   8399.60      779.24
3   50            40752.0   1630.08         NaN
    100           62991.0   2519.64      889.56
    150           71784.0   2871.36      351.72
    200           81803.0   3272.12      400.76
    250           91105.0   3644.20      372.08
    300          130836.0   5233.44     1589.24
    350          186180.0   7447.20     2213.76
    400          245664.0   9826.56     2379.36
    450          305296.0  12211.84     2385.28
    500          342102.0  13684.08     1472.24
4   50             6155.0    246.20         NaN
    100            9286.0    371.44      125.24
    150           14951.0    598.04      226.60
    200           42150.0   1686.00     1087.96
    250           68332.0   2733.28     1047.28
    300          118009.0   4720.36     1987.08
    350          150034.0   6001.36     1281.00
    400          198227.0   7929.08     1927.72
    450          244773.0   9790.92     1861.84
    500          301020.0  12040.80     2249.88
5   200            5024.0    200.96         NaN
    250           10615.0    424.60      223.64
    300           32163.0   1286.52      861.92
    350           54774.0   2190.96      904.44
    400           85604.0   3424.16     1233.20
    450          107992.0   4319.68      895.52
    500          126579.0   5063.16      743.48
6   200            4989.0    199.56         NaN
    250           10669.0    426.76      227.20
    300           32194.0   1287.76      861.00
    350           54915.0   2196.60      908.84
    400           85721.0   3428.84     1232.24
    450          108137.0   4325.48      896.64
    500          126679.0   5067.16      741.68

In [23]:
#Are_Urb["Area_Dif_U"] = Are_Urb.fillna(Are_Urb.Area_KM2)
#Are_Urb.groupby(level = 0).Area_Dif_U.fillna(Are_Urb.Area_KM2)
Are_Urb["Area_Dif_U"] = Are_Urb["Area_Dif_U"].fillna(Are_Urb['Area_KM2'])
Area_Net_Def["Area_Dif_NDEF"] = Area_Net_Def["Area_Dif_NDEF"].fillna(Area_Net_Def['Area_KM2'])
Are_Def_Sum["Area_Dif_DEF"] = Are_Def_Sum["Area_Dif_DEF"].fillna(Are_Def_Sum['Area_KM2'])
Area_Net_Def

Area_KM2  Area_Dif_NDEF
Dam Buffer_Zone                          
1   50             2971.28        2971.28
    100           10822.84        7851.56
    150           19596.16        8773.32
    200           31783.04       12186.88
    250           50650.12       18867.08
    300           65325.72       14675.60
    350           80794.32       15468.60
    400          101390.24       20595.92
    450          131542.80       30152.56
    500          168094.76       36551.96
2   50             2967.24        2967.24
    100            9032.60        6065.36
    150           12052.48        3019.88
    200           23268.64       11216.16
    250           38649.92       15381.28
    300           46672.12        8022.20
    350           53432.84        6760.72
    400           71538.32       18105.48
    450          100327.68       28789.36
    500          126002.64       25674.96
3   50            -2551.48       -2551.48
    100           -8563.28       -6011.80
    150          -11487.68       -2924.40
    200          -17235.08       -5747.40
    250          -36062.16      -18827.08
    300          -50757.36      -14695.20
    350          -74589.56      -23832.20
    400          -98247.32      -23657.76
    450         -127218.52      -28971.20
    500         -156050.64      -28832.12
4   50              786.84         786.84
    100           -1760.80       -2547.64
    150           -6306.32       -4545.52
    200          -17558.52      -11252.20
    250          -30611.28      -13052.76
    300          -49101.24      -18489.96
    350          -69042.68      -19941.44
    400          -89211.68      -20169.00
    450         -117316.76      -28105.08
    500         -143613.12      -26296.36
5   50            -2714.16       -2714.16
    100           -9358.68       -6644.52
    150          -19723.12      -10364.44
    200          -35514.72      -15791.60
    250          -53863.16      -18348.44
    300          -72106.88      -18243.72
    350         -101340.60      -29233.72
    400         -133380.04      -32039.44
    450         -169058.16      -35678.12
    500         -210129.80      -41071.64
6   50            -2704.52       -2704.52
    100           -9345.28       -6640.76
    150          -19701.36      -10356.08
    200          -35512.80      -15811.44
    250          -53849.44      -18336.64
    300          -72101.40      -18251.96
    350         -101346.68      -29245.28
    400         -133381.20      -32034.52
    450         -169073.68      -35692.48
    500         -210153.04      -41079.36

In [28]:
#Waste
#Area_Net_Def.groupby(level=0)["Area_Dif_NDEF"].iloc[0] =Are_Def_Sum.groupby(level=0)["Area_KM2"].iloc[0]
#Are_Urb.groupby(level=0)["Area_Dif_U"].iloc[0] = Are_Urb.groupby(level=0)["Area_KM2"].iloc[0]
#Are_Def_Sum.groupby(level=0)["Area_Dif_DEF"].iloc[0] =Area_Net_Def.groupby(level=0)["Area_KM2"].iloc[0]

In [24]:
Sum_A = pd.DataFrame(data =Area_Net_Def["Area_Dif_NDEF"] )
#Sum_A["Area_Dif_DEF"] = Are_Def_Sum["Area_Dif_DEF"] 
Sum_A["Area_Dif_U"] = Are_Urb["Area_Dif_U"] 
for col in Sum_A.columns: 
    print(col)
    Sum_A[f"{col}_Percntage"] = abs(Sum_A[col] *100)/ Sum_A.groupby(level  = 0 )[col].apply(lambda c: c.abs().sum())
Sum_A = Sum_A.reset_index()
Sum_A.to_excel(rf'D:\mskenawi\one drive\OneDrive - NTNU\PhD\LUHP\Summary\Summary of Data\LT_Buf_Intk\{OP_N}.xlsx')

Area_Dif_NDEF
Area_Dif_U


In [30]:
Sum_A

,Dam,Buffer_Zone,Area_Dif_NDEF,Area_Dif_U,Area_Dif_NDEF_Percntage,Area_Dif_U_Percntage
0,1,50,2961.28,535.60,1.761833,21.107888
1,1,100,7851.00,505.28,4.671003,19.912983
2,1,150,8769.28,210.32,5.217340,8.288669
3,1,200,12189.88,184.40,7.252448,7.267167
4,1,250,18890.04,158.48,11.238752,6.245665
5,1,300,14674.56,160.12,8.730725,6.310297
6,1,350,15468.52,286.80,9.203096,11.302730
7,1,400,20609.64,168.16,12.261839,6.627152
8,1,450,30155.84,173.48,17.941412,6.836812
9,1,500,36509.48,154.80,21.721552,6.100637


In [31]:
Sum_A_P = Sum_A
for col in Sum_A_P.columns: 
    print(col)
    #Sum_A_P[col].groupby(level= 0)  =  (Sum_A_P[col].groupby(level= 0))*100 
    Sum_A_P[col] = Sum_A_P[col] *100/ Sum_A_P.groupby(level  = 0 )[col].transform('sum')
Sum_A_P

Dam


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [774]:
#Absolute Plotting 
fig, ax = plt.subplots(figsize=(11.69,8.27))
#Sum_A_P.plot(ax=ax, marker='o', ls='-')
ax.set_xlabel("BufferZone 100 M")
ax.set_ylabel(r"Change in Km$^2$")
Sum_A.plot(kind='bar', ax=ax, color = ["#1c4c5e", "#a6e590", "#060369"])
ax.legend(["Net Deforestation", "Gross Deforestation", "Urbanization"])
plt.show()
#plt.savefig(r"C:\Users\mskenawi\Dropbox\PhD\LU_HP\Ecognition\WorkSpaces\Trollheim\Output\Absolute change of Intakes_500m.png",dpi=1200)

In [1033]:
#Percent Plotting 
fig, ax = plt.subplots(figsize=(11.69,8.27))
#Sum_A_P.plot(ax=ax, marker='o', ls='-')
ax.set_xlabel("BufferZone 100 M")
ax.set_ylabel("Percentage of Change %")
Sum_A_P["Area_Dif_U"].plot(kind='bar', ax=ax)
#ax.legend(["Net Deforestation", "Gross Deforestation", "Urbanization"])
plt.show()

In [1036]:
fig, ax = plt.subplots(figsize=(11.69,8.27))
plot = meansum.plot(kind='bar',yerr=stdsum,colormap='OrRd_r',edgecolor='black',grid=False,figsize=8,2),ax=ax,position=0.45,error_kw=dict(ecolor='black',elinewidth=0.5, lolims=True),width=0.8)
for ch in plot.get_children():
    if str(ch).startswith('Line2D'): # this is silly, but it appears that the first Line in the children are the caplines...
        ch.set_marker('_')
        ch.set_markersize(10) # to change its size
        break
plt.show()

SyntaxError: invalid syntax (<string>, line 3)

In [ ]:

fig, ax = plt.subplots()
df = pd.DataFrame({"val":[1,2,3,4],"error":[.4,.3,.6,.9]})
meansum = df["val"]
stdsum = df["error"]

plot = meansum.plot(kind='bar',yerr=stdsum,colormap='OrRd_r',edgecolor='black',grid=False,figsize=8,2),ax=ax,position=0.45,error_kw=dict(ecolor='black',elinewidth=0.5, lolims=True),width=0.8)
for ch in plot.get_children():
    if str(ch).startswith('Line2D'): # this is silly, but it appears that the first Line in the children are the caplines...
        ch.set_marker('_')
        ch.set_markersize(10) # to change its size
        break
plt.show()

In [1039]:
arcpy.ia.CreateAccuracyAssessmentPoints("Troll_T3", r"C:\Users\mskenawi\Dropbox\PhD\LU_HP\Ecognition\WorkSpaces\Trollheim\GIS\TrollHeim\TrollHeim.gdb\ACC_T3", "CLASSIFIED", 400, "EQUALIZED_STRATIFIED_RANDOM", None)

ExecuteError: ERROR 160385: Workspace or data source is read only.
Workspace or data source is read only.
Workspace or data source is read only.
Failed to execute (CreateAccuracyAssessmentPoints).
